<a href="https://colab.research.google.com/github/phutares/Spamfiltering/blob/main/SpamFiltering_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Import Library**

In [ ]:
from sklearn.model_selection import train_test_split

import pandas as pd
import re

# สำหรับทำ classify และทดสอบโมเดล
from nltk import FreqDist, precision, recall, f_measure, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
import nltk
import numpy as np



In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#**Import File**

In [ ]:
pandasFile = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spam.csv',engine='python')

In [ ]:
pandasFile

v1  ... Unnamed: 4
0      ham  ...        NaN
1      ham  ...        NaN
2     spam  ...        NaN
3      ham  ...        NaN
4      ham  ...        NaN
...    ...  ...        ...
5567  spam  ...        NaN
5568   ham  ...        NaN
5569   ham  ...        NaN
5570   ham  ...        NaN
5571   ham  ...        NaN

[5572 rows x 5 columns]

In [ ]:
df = pd.DataFrame(pandasFile)

#**Data Preparation**

In [ ]:
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1)

In [ ]:
df.isna()

v1     v2
0     False  False
1     False  False
2     False  False
3     False  False
4     False  False
...     ...    ...
5567  False  False
5568  False  False
5569  False  False
5570  False  False
5571  False  False

[5572 rows x 2 columns]

In [ ]:
df.isna().sum() 

v1    0
v2    0
dtype: int64

In [ ]:
df = df.rename(columns={"v1":"label", "v2":"text"})
df.head(2)

label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...

In [ ]:
df_ham = df[df['label'] == 'ham'].copy()
df_spam = df[df['label'] == 'spam'].copy()

#**Data Cleansing and Preprocessing**

In [ ]:
def clean_message(msgs, pos):
    """ 
    Clean the message
    1.Remove all non-words
    2.transform words into lower case
    3.remove stop words
    4.perform stemming
    """
    msgs = re.sub("[^A-Za-z]", " ", msgs)
    msgs = msgs.lower()
    msgs = nltk.word_tokenize(msgs)
    lemmatizer = WordNetLemmatizer()
    msgs = [lemmatizer.lemmatize(msg,pos) for msg in msgs if not msg in stopwords.words('english')]
    msgs = " ".join(msgs)
    return msgs

In [ ]:
corpus = []

for i in range(0, len(df)):
    msgs = clean_message(df.text[i], pos = wordnet.VERB)
    corpus.append(msgs)

In [ ]:
from collections import Counter

df_new = pd.DataFrame({'Label':df['label'], 'text': corpus})

In [ ]:
df_new.head()


Label                                               text
0   ham  go jurong point crazy available bugis n great ...
1   ham                              ok lar joke wif u oni
2  spam  free entry wkly comp win fa cup final tkts st ...
3   ham                u dun say early hor u c already say
4   ham                nah think go usf live around though

# **Bag of words with CountVectorizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
y = df_new['Label']

In [ ]:
print(type(y))

<class 'pandas.core.series.Series'>


In [ ]:
print(corpus)

['go jurong point crazy available bugis n great world la e buffet cine get amore wat', 'ok lar joke wif u oni', 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply', 'u dun say early hor u c already say', 'nah think go usf live around though', 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv', 'even brother like speak treat like aid patent', 'per request melle melle oru minnaminunginte nurungu vettam set callertune callers press copy friends callertune', 'winner value network customer select receivea prize reward claim call claim code kl valid hours', 'mobile months u r entitle update latest colour mobiles camera free call mobile update co free', 'gon na home soon want talk stuff anymore tonight k cry enough today', 'six chance win cash pound txt csh send cost p day days tsandcs apply reply hl info', 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw', 'search rig

In [ ]:
print(X[0])

[0 0 0 ... 0 0 0]


In [ ]:
X.shape

(5572, 6578)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
y_test

4456    ham
690     ham
944     ham
3768    ham
1189    ham
       ... 
2906    ham
1270    ham
3944    ham
2124    ham
253     ham
Name: Label, Length: 1115, dtype: object

In [ ]:
print(type(X_test))

<class 'numpy.ndarray'>


In [ ]:
print(X_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


#**Train The Classifier**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)


#**Test And Evaluate**

In [ ]:
Z


In [ ]:
y_pred

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [ ]:
print(X_train[4:5].shape)

(1, 6578)


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy : ", accuracy)

Accuracy :  0.9775784753363229


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         ham       0.99      0.98      0.99       949
        spam       0.91      0.95      0.93       166

    accuracy                           0.98      1115
   macro avg       0.95      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [ ]:
print(cv)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [ ]:
examples = ["Bob ask you for a book?", "free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply"]
example_counts = cv.transform(examples)
print(example_counts)

  (0, 351)	1
  (0, 655)	1
  (0, 668)	1
  (1, 284)	1
  (1, 1108)	1
  (1, 1283)	1
  (1, 1771)	2
  (1, 1894)	2
  (1, 1991)	1
  (1, 2107)	1
  (1, 3424)	1
  (1, 4540)	1
  (1, 4584)	1
  (1, 4625)	1
  (1, 5347)	1
  (1, 5375)	1
  (1, 5680)	1
  (1, 5797)	1
  (1, 5957)	1
  (1, 6346)	1
  (1, 6380)	1


In [ ]:
predictions = spam_detect_model.predict(example_counts)
print(predictions)

['ham' 'spam']
